In [1]:
import tensorflow as tf
import numpy as np
import json
import pickle
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image as PIL_Image
from io import BytesIO as BY
import requests
import cgi
import os
from math import sin, cos, sqrt, atan2, radians



In [10]:
load = requests.get("https://hear-seoul.herokuapp.com/api/get-spot-data?key=sadfhlfhqoiuahdfasdfhjq")
load =load.json()

In [11]:
img = load['results'][0]['img_src'][0]
image = requests.get(img).content
name = os.path.basename(img)
with open('Data/'+name, 'wb') as f:
    f.write(image)

In [12]:
count = load['totalCount']

In [13]:
dataset = []


    #이미지 다운로드 및 로드
for num in range(count):

    img_count = load['results'][num]['img_src']
    for img_id in img_count:
        img = img_id
        image = requests.get(img).content
        name = os.path.basename(img)
        with open('Data/'+name, 'wb') as f:
            f.write(image)
            
        img = PIL_Image.open("Data/"+name)
        img = img.resize((224,224))
        img = list(np.asarray(img))
        img_t = np.array([img], dtype = np.float32)
        dataset.append(img_t)

In [14]:
dataset = []
img_label = []


    #이미지 로드
for num in range(count-1):

    img_count = load['results'][num]['img_src']
    for img_id in img_count:
        img = img_id
        name = os.path.basename(img)
        img = PIL_Image.open("Data/"+name)
        img = img.resize((224,224))
        img = list(np.asarray(img))
        img_t = np.array([img], dtype = np.float32)
        dataset.append(img_t)
        img_label.append(num)

In [15]:
image_count = np.shape(dataset)[0]

In [16]:
#STARTING VGG-16


# X = SHAPE (NONE, HEIGHT, WIDTH, CHANNELS)
X = tf.placeholder(shape = (None, 224,224, 3), dtype = tf.float32)


#VGG -16 START
conv1 = tf.layers.conv2d(X, filters = 64, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv2 = tf.layers.conv2d(conv1, filters = 64, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)

pool1 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides = [1,2,2,1], padding="VALID")


#FIRST END


conv3 = tf.layers.conv2d(pool1, filters = 128, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv4 = tf.layers.conv2d(conv3, filters = 128, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)

pool2 = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides = [1,2,2,1], padding="VALID")


#SECOND END

conv5 = tf.layers.conv2d(pool2, filters = 256, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv6 = tf.layers.conv2d(conv5, filters = 256, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)

pool3 = tf.nn.max_pool(conv6, ksize=[1,2,2,1], strides = [1,2,2,1], padding="VALID")


#THIRD END


conv7 = tf.layers.conv2d(pool3, filters = 512, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv8 = tf.layers.conv2d(conv7, filters = 512, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv9 = tf.layers.conv2d(conv8, filters = 512, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)

pool4 = tf.nn.max_pool(conv9, ksize=[1,2,2,1], strides = [1,2,2,1], padding="VALID")


#FOURTH END



conv10 = tf.layers.conv2d(pool4, filters = 512, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv11 = tf.layers.conv2d(conv10, filters = 512, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)
conv12 = tf.layers.conv2d(conv11, filters = 512, kernel_size = 3, strides = [1,1], padding="SAME",
                       activation = tf.nn.relu)


#VGG -16 END


In [17]:
init = tf.global_variables_initializer()
#saver = tf.train.Saver(tf.global_variables())

feature = []

#VGG-16 RUN
with tf.Session() as sess:
    
    
    with tf.device("/gpu:0"):
        init.run()
        for id in range(np.shape(dataset)[0]):
            features = sess.run(conv12, feed_dict={X: dataset[id]})
            
            feature.append(features)
            #saver.save(sess, 'VGG_model/VGG.module')
            

In [18]:
X_array= np.asarray(feature, dtype=np.float32)
#X_array=feature

In [32]:
#PCA
#충분한 분산이 될때까지 더해야 할 차원 수 선택
#sess = tf.Session()
#with sess.as_default():
#    X_train=conv12.eval()
#feats = normalize(features)
X_reduced=[]
dimention_count=2
for i in range(image_count):
    tmp=X_array[i][0]
    X=tmp.reshape(14,7168)# 14*14*512  => 14*7168  2dimention input require
    pca=PCA(n_components=dimention_count)
    X_reduced.append(pca.fit_transform(X))


In [30]:
dArray=np.asarray(X_reduced, dtype=np.float32) #(image_count,14,1)
dArray.shape

(216, 14, 2)

In [33]:
dArray=np.asarray(X_reduced, dtype=np.float32) #(image_count,14,1)
X_reduced2=dArray.reshape(image_count,14*dimention_count)
#X_reduced2.shape

In [34]:
X_reduced2.shape

(216, 28)

In [36]:


#kmeans
# k개수 지정 k=(sqrt n/2) n은 데이터의 개수

    # 모든 데이터를 상수 텐서로 옮김

#k=np.sqrt(np.size(X_reduced2)/2).astype('int')
# change to k=5
k=5
vectors = tf.constant(X_reduced2)

centroides = tf.Variable(tf.slice(tf.random_shuffle(vectors),[0,0],[k,-1]))

expanded_vectors = tf.expand_dims(vectors, 0)
expanded_centroides = tf.expand_dims(centroides, 1)

diff = tf.subtract(expanded_vectors, expanded_centroides)
sqr = tf.square(diff)
distances = tf.reduce_sum(sqr, 2)
assignments = tf.argmin(distances,0)

means = tf.concat([tf.reduce_mean(tf.gather(vectors, tf.reshape( tf.where(tf.equal(assignments, c)) ,[1, -1])), reduction_indices=[1]) for c in range(k)],0)

update_centroides = tf.assign(centroides, means)


init_op = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init_op)

for step in range(100):
    _, centroid_values, assignment_values = sess.run([update_centroides, centroides, assignments])
    
    


In [37]:
    
#knn

knn = KNeighborsClassifier(k)
knn.fit(X_reduced2, assignment_values)



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [39]:
X_reduced2.shape

(216, 28)

In [40]:
#X_reduced2.shape
temp=X_reduced2#[1].reshape(1,14)
y_pred = knn.predict(temp)


In [41]:
"""
# 에측 값과 같은 라벨을 가진, 인덱스를 저장

#예측할 값
Test = 0

#라벨
Target = y_pred[Test]

#훈련된 라벨
True_label = assignment_values.tolist()

#같은 라벨을 가지고 있는 값의 인덱스를 찾아 저장
Target_label = [i for i,x in enumerate(True_label) if x == Target]

Target_label"""

[0,
 2,
 4,
 6,
 7,
 11,
 12,
 15,
 25,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 43,
 44,
 58,
 66,
 74,
 75,
 77,
 78,
 79,
 84,
 90,
 91,
 100,
 106,
 108,
 119,
 120,
 123,
 124,
 125,
 126,
 130,
 136,
 144,
 154,
 167,
 171,
 174,
 175,
 177,
 178,
 180,
 181,
 185,
 186,
 187,
 191,
 195,
 212,
 214]

In [42]:
"""
#해당 값과 예측된 값의 지도상의 좌표를 가져와, 유클리디언 거리를 측정

#지구의 반지름
R = 6371

#거리 저장 변수
PD = []

#테스트 데이터 GPS정보
Test_la = radians(load['results'][Test]['location']['latitude'])
Test_lo = radians(load['results'][Test]['location']['longitude'])

#각 같은 LABEL별로 유클리디언 거리 측정 후 저장
for label in Target_label:
    num = img_label[label]
    Train_la = radians(load['results'][num]['location']['latitude'])
    Train_lo = radians(load['results'][num]['location']['longitude'])
    
    dlon = Train_lo - Test_lo
    dlat = Train_la - Test_la
    
    a = sin(dlat / 2) **2 + cos(Test_la)*cos(Train_la)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    distance = R * c
    
    PD.append(distance)
    
#가장 작은 거리 찾기
ANS = PD.index(min(PD))
ANS = img_label[Target_label[ANS]]
"""

In [19]:
"""
Result = {"recommand" : 
          [
              {"ID" : load['results'][Test]['objectId']},
              {"Suggest" : load['results'][ANS]['objectId']}
          ]
         }
json.dumps(Result)

with open('Result.json','w',encoding="utf-8") as mkfil:
    json.dump(Result, mkfil)
    
for i in range(count):
    payload={"id":load['results'][i]['id'],
        "suggest":load['results'][i]['id']}
    requests.get('https://hear-seoul.herokuapp.com/api/suggestion-upload', params=payload)
    
"""

In [43]:
# 에측 값과 같은 라벨을 가진, 인덱스를 저장
for i in range(count):
    #예측할 값
    Test = i

    #라벨
    Target = y_pred[Test]

    #훈련된 라벨
    True_label = assignment_values.tolist()

    #같은 라벨을 가지고 있는 값의 인덱스를 찾아 저장
    Target_label = [i for i,x in enumerate(True_label) if x == Target]

    #Target_label
    #해당 값과 예측된 값의 지도상의 좌표를 가져와, 유클리디언 거리를 측정

    #지구의 반지름
    R = 6371

    #거리 저장 변수
    PD = []

    #테스트 데이터 GPS정보
    Test_la = radians(load['results'][Test]['location']['latitude'])
    Test_lo = radians(load['results'][Test]['location']['longitude'])

    #각 같은 LABEL별로 유클리디언 거리 측정 후 저장
    for label in Target_label:
        num = img_label[label]
        Train_la = radians(load['results'][num]['location']['latitude'])
        Train_lo = radians(load['results'][num]['location']['longitude'])

        dlon = Train_lo - Test_lo
        dlat = Train_la - Test_la

        a = sin(dlat / 2) **2 + cos(Test_la)*cos(Train_la)*sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))

        distance = R * c

        PD.append(distance)

    #가장 작은 거리 찾기
    ANS = PD.index(min(PD))
    ANS = img_label[Target_label[ANS]]

'''
    Result = {"recommand" : 
              [
                  {"ID" : load['results'][Test]['objectId']},
                  {"Suggest" : load['results'][ANS]['objectId']}
              ]
             }
    json.dumps(Result)

    with open('Result.json','w',encoding="utf-8") as mkfil:
        json.dump(Result, mkfil)
  '''  
    payload={"id":load['results'][i]['id'],
        "suggest":load['results'][ANS]['id']}
    requests.get('https://hear-seoul.herokuapp.com/api/suggestion-upload?key=sadfhlfhqoiuahdfasdfhjq', params=payload)
    